## Máster en Big Data y Data Science

### Metodologías de gestión y diseño de proyectos de big data

#### AP2 - Preparación de los datos

---

En esta libreta se realizan las transforamciones sobre los datasets del escenario en función 
de los resultados de la verificación de calidad de datos. 

---

In [1]:
#Se importan las librerias a utilizar

import pandas as pd

----

##### Lectura de los datasets

In [2]:
df_creditos = pd.read_csv("../../data/processed/datos_creditos_mc.csv", sep=";")
display(df_creditos.head(1))

df_tarjetas = pd.read_csv("../../data/processed/datos_tarjetas_mc.csv", sep=";")
display(df_tarjetas.head(1))

df_integrado = pd.read_csv("../../data/processed/datos_integrados.csv", sep=";")
display(df_integrado.head(1))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,completitud_fila,situacion_vivienda_ok,objetivo_credito_ok,falta_pago_ok
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,0.0,True,True,True


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo,regla_pct_ingresos,ingreso_regla2
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0,ok,Aprobado


---
#### Aplicación de transformaciones

**Operaciones a realizar**

1. Selección de columnas
2. Filtrado de filas
3. Construcción de atributos
4. Integración de datasets
5. Formateo definitivo


----

Selección de datos

Al realizar un análisis exploratorio, se pudo rescatar ciertas cosas como:
- Se tiene la hipótesis que "nivel_educativo" no interviene en el número de operaciones realizadas.
- El nivel de la tarjeta no parece tener relación con "nivel_credito_tc" ni con "nivel_educativo".
- Se toma en consideración el análisis inicial (Descripción de los datos) para identificar que en la columna "nivel_tarjeta" tiene más del 90% de los datos el valor categórico "Blue", lo que indica que es una tarjeta común. Es así que se considera que la mejor manera de seccionar es a través de el límite crediticio (limite_credito_tc).
- Por los motivos anteriores se decide eliminar las columnas "nivel_educativo" y "nivel_tarjeta".

In [3]:
#El nivel de tarjeta con más usuarios es "Blue"
grouped = df_tarjetas.groupby('nivel_tarjeta')['nivel_educativo'].value_counts()

print(grouped)

nivel_tarjeta  nivel_educativo         
Blue           UNIVERSITARIO_COMPLETO      2899
               UNIVERSITARIO_INCOMPLETO    2331
               SECUNDARIO_COMPLETO         1888
               DESCONOCIDO                 1420
               POSGRADO_INCOMPLETO          476
               POSGRADO_COMPLETO            422
Gold           UNIVERSITARIO_COMPLETO        36
               SECUNDARIO_COMPLETO           26
               UNIVERSITARIO_INCOMPLETO      26
               DESCONOCIDO                   18
               POSGRADO_INCOMPLETO            5
               POSGRADO_COMPLETO              5
Platinum       UNIVERSITARIO_COMPLETO         8
               POSGRADO_INCOMPLETO            3
               UNIVERSITARIO_INCOMPLETO       3
               DESCONOCIDO                    3
               POSGRADO_COMPLETO              2
               SECUNDARIO_COMPLETO            1
Silver         UNIVERSITARIO_COMPLETO       185
               UNIVERSITARIO_INCOMPLETO     140


In [4]:
# Columnas a eliminar
col_eliminar_creditos = ['completitud_fila', 'situacion_vivienda_ok', 'objetivo_credito_ok', 'falta_pago_ok'] 
col_eliminar_tarjetas = ['nivel_tarjeta', 'nivel_educativo']
col_eliminar_integrado = ['regla_pct_ingresos', 'ingreso_regla2']

# Se ejecuta la operación
df_creditos.drop(col_eliminar_creditos, inplace=True, axis=1)
df_tarjetas.drop(col_eliminar_tarjetas, inplace=True, axis=1)
df_integrado.drop(col_eliminar_integrado, inplace=True, axis=1)

In [5]:
print("Vista del dataset de datos de créditos:")
display(df_creditos.head(1))

print("Vista del dataset de datos de tarjetas:")
display(df_tarjetas.head(1))

print("Vista del dataset de datos integrados:")
display(df_integrado.head(1))

Vista del dataset de datos de créditos:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N


Vista del dataset de datos de tarjetas:


,id_cliente,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,708082083.0,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0


Vista del dataset de datos integrados:


,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,nivel_educativo,nivel_tarjeta,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,36.0,CASADO,ACTIVO,15149.0,F,3544.0,SECUNDARIO_COMPLETO,Blue,111.0,3.0


Limpieza de los datos: Filtrado a nivel de filas

- Filtro para reglas de negocio

In [6]:
#Se puede definir una función para aplicar los cálculos
def regla_pct_ingresos_credito(row):
    pct_ingreso = row.pct_ingreso
    ingresos = row.ingresos
    
    if pct_ingreso > 0.5 and ingresos <= 20000:
        # Es un error, no cumple la regla definida
        return 'err'
    else:
        return 'ok'


# Se aplica la función para todos los elementos del dataset
regla_pct_ingresos = df_creditos.apply(lambda row: regla_pct_ingresos_credito(row), axis=1).rename("regla_pct_ingresos")

# Se unen los resultados al dataset inicial
df_creditos = pd.concat([df_creditos, regla_pct_ingresos], axis=1)
df_creditos.head(5)

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,0,N,ok
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,1,Y,ok
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,0,Y,ok
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,0,N,ok
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,0,N,ok


In [7]:
def verificar_credito(row):
    duracion_credito = row['duracion_credito']
    ingresos = row['ingresos']
    importe_solicitado = row['importe_solicitado']
    situacion_vivienda = row['situacion_vivienda']

    duracion_minima = df_creditos['duracion_credito'].min()

    if duracion_credito == duracion_minima:
        porcentaje_ingresos = (importe_solicitado/ingresos) * 100

        if situacion_vivienda == 'PROPIA':
            return "Aprobado"
        elif porcentaje_ingresos <= 60:
            return "Aprobado"
        else:
            return "Rechazado"
    else:
        return "Aprobado"

#PRUEBA_REGLA_2 = df_creditos.copy()

# Aplicar la función a cada fila del DataFrame
df_creditos['ingreso_regla2'] = df_creditos.apply(verificar_credito, axis=1)

# Se visualizan los datos
print("Se visualizan las tuplas que no cumplen con la regla:\n")
display(df_creditos[df_creditos.ingreso_regla2 == 'Rechazado'].head())

Se visualizan las tuplas que no cumplen con la regla:



,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,estado_credito,falta_pago,regla_pct_ingresos,ingreso_regla2
466,709040508.0,21,15000,2,0.0,HIPOTECA,19500,EDUCACIÓN,0.77,9.64,1,N,err,Rechazado
541,709186983.0,23,20000,2,1.0,ALQUILER,32900,EDUCACIÓN,0.61,16.00,1,N,ok,Rechazado
1646,711399408.0,24,29100,2,8.0,HIPOTECA,46000,PERSONAL,0.63,12.99,1,Y,ok,Rechazado
2573,713115483.0,22,7000,2,0.0,ALQUILER,10000,MEJORAS_HOGAR,0.70,12.21,1,N,err,Rechazado
3041,713962233.0,22,30000,2,1.0,ALQUILER,48000,EDUCACIÓN,0.63,18.39,1,N,ok,Rechazado


In [8]:
# Se filtran las filas con algún error detectado
print(f"Filas antes del filtro: {df_creditos.shape[0]}")

temp = df_creditos[df_creditos['edad'] < 90]
temp_c = temp[temp['antiguedad_empleado'] <= 50] # conservar empleados con menor o igual a 50 años de antigüedad
temp_c2 = temp_c[temp_c['regla_pct_ingresos'] == 'ok'] # conservar filas con cumplimiento de regla 1
temp_c3 = temp_c2[temp_c2['ingreso_regla2'] == 'Aprobado'] # conservar filas con cumplimiento de regla 2
print(f"Filas despues del filtro: {temp_c3.shape[0]}")

Filas antes del filtro: 10127
Filas despues del filtro: 9765


- Relleno de nulos

In [17]:
promedio = temp_c3['tasa_interes'].mean()
temp_c3['tasa_interes'].fillna(promedio, inplace=True)s

C:\Users\Asus_i5\AppData\Local\Temp\ipykernel_16664\3474375956.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_c3['tasa_interes'].fillna(promedio, inplace=True)


Se comprueba los nulos en cada columna

In [18]:
nulos=temp_c3.isnull().sum()
print(nulos)

id_cliente             0
edad                   0
importe_solicitado     0
duracion_credito       0
antiguedad_empleado    0
situacion_vivienda     0
ingresos               0
objetivo_credito       0
pct_ingreso            0
tasa_interes           0
estado_credito         0
falta_pago             0
regla_pct_ingresos     0
ingreso_regla2         0
dtype: int64


Integración de datos

In [19]:
df_integrado = pd.merge(temp_c3, df_tarjetas, on='id_cliente', how='inner')
coincidencias = df_integrado.shape[0]

print(f"Filas del dataset integrados con los filtros previamente realizados: {coincidencias}")

Filas del dataset integrados con los filtros previamente realizados: 9765


In [20]:
print(f"Cantidad de columnas del dataset integrado: {df_integrado.shape[1]}")

Cantidad de columnas del dataset integrado: 22


In [21]:
display(df_integrado.head(5))

,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,tasa_interes,...,regla_pct_ingresos,ingreso_regla2,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,5.79,...,ok,Aprobado,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0
1,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,15.99,...,ok,Aprobado,45.0,SOLTERO,PASIVO,992.0,M,3421.0,21.0,0.0
2,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,12.72,...,ok,Aprobado,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,23.0,3.0
3,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,8.00,...,ok,Aprobado,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,82.0,2.0
4,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,7.74,...,ok,Aprobado,41.0,CASADO,ACTIVO,4369.0,F,3128.0,59.0,2.0


Transformación de atributos

In [23]:
# Columna: estado_civil
cambios_estado_civil = {
    'CASADO' : 'C',
    'SOLTERO' : 'S',
    'DESCONOCIDO' : 'N',
    'DIVORCIADO' : 'D',
}

estado_civil_N = df_integrado.loc[:, ('estado_civil')].map(cambios_estado_civil).rename('estado_civil_N')
df_final = pd.concat([estado_civil_N, df_integrado], axis=1)
df_final.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,pct_ingreso,...,ingreso_regla2,regla_pct_ingresos,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,C,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,0.17,...,Aprobado,ok,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0
1,S,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,0.05,...,Aprobado,ok,45.0,SOLTERO,PASIVO,992.0,M,3421.0,21.0,0.0
2,D,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,0.25,...,Aprobado,ok,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,23.0,3.0
3,S,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,0.26,...,Aprobado,ok,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,82.0,2.0
4,C,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,0.20,...,Aprobado,ok,41.0,CASADO,ACTIVO,4369.0,F,3128.0,59.0,2.0


In [24]:
# Columna: estado_credito
cambios_estado_credito = {
    0 : 'Cancelado',
    1 : 'Activo',

}

estado_credito_N = df_integrado.loc[:, ('estado_credito')].map(cambios_estado_credito).rename('estado_credito_N')
df_final = pd.concat([estado_civil_N, estado_credito_N, df_integrado], axis=1)
df_final.head()

# Sobre este resultado será necesario eliminar las columnas auxiliares

,estado_civil_N,estado_credito_N,id_cliente,edad,importe_solicitado,duracion_credito,antiguedad_empleado,situacion_vivienda,ingresos,objetivo_credito,...,ingreso_regla2,regla_pct_ingresos,antiguedad_cliente,estado_civil,estado_cliente,gastos_ult_12m,genero,limite_credito_tc,operaciones_ult_12m,personas_a_cargo
0,C,Cancelado,708082083.0,24,11000,3,5.0,HIPOTECA,64800,INVERSIONES,...,Aprobado,ok,36.0,CASADO,ACTIVO,15149.0,F,3544.0,111.0,3.0
1,S,Activo,708083283.0,24,1500,2,0.0,ALQUILER,30996,MEJORAS_HOGAR,...,Aprobado,ok,45.0,SOLTERO,PASIVO,992.0,M,3421.0,21.0,0.0
2,D,Cancelado,708084558.0,23,10000,2,7.0,OTROS,40104,EDUCACIÓN,...,Aprobado,ok,38.0,DIVORCIADO,PASIVO,1447.0,M,8258.0,23.0,3.0
3,S,Cancelado,708085458.0,25,6000,4,2.0,ALQUILER,23198,INVERSIONES,...,Aprobado,ok,24.0,SOLTERO,ACTIVO,3940.0,F,1438.3,82.0,2.0
4,C,Cancelado,708086958.0,26,10000,2,0.0,HIPOTECA,50000,EDUCACIÓN,...,Aprobado,ok,41.0,CASADO,ACTIVO,4369.0,F,3128.0,59.0,2.0


Exportación de metadatos